# This notebook demonstrates to transform the output of CLUMPY to a template we can use for the HAWC analysis

In [1]:
# import the necessary packages
import astropy.io.fits as fits
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np
import matplotlib.pyplot as plt # may not be necessary
import matplotlib.colors as colors
import healpy as hp
from scipy.interpolate import interp2d
import pickle
import os

from matplotlib import rcParams
rcParams['figure.figsize'] = [8.0, 6.0]
rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14
rcParams['axes.labelsize'] = 14
rcParams['legend.fontsize'] = 11

In [2]:
# M87
lon_M87 = 283.77
lat_M87 = 74.49
# M49
lon_M49 = 286.92
lat_M49 = 70.20
# Virgo Cluster
lon_Virgo = 283.77
lat_Virgo = 74.49

M87_coords = SkyCoord(lon_M87*u.degree, lat_M87*u.degree, frame='galactic', unit='degree')
M87_lat = M87_coords.galactic.l.radian
M87_lon = M87_coords.galactic.b.radian

M49_coords = SkyCoord(lon_M49*u.degree, lat_M49*u.degree, frame='galactic', unit='degree')
M49_lat = M49_coords.galactic.l.radian
M49_lon = M49_coords.galactic.b.radian
"""
M87_filename    = "../output/decay_M872D_FOVdiameter12.0deg_rse1_alphaint0.11deg_nside1024.fits"
M49_filename    = "../output/decay_M492D_FOVdiameter12.0deg_rse1_alphaint0.11deg_nside1024.fits"
VirgoC_filename = "../output/decay_VirgoC2D_FOVdiameter12.0deg_rse1_alphaint0.11deg_nside1024.fits"
"""
M87_filename    = "../output/decay_M87_NFW2D_FOVdiameter14.0deg_rse1_alphaint0.06deg_nside2048.fits"
M49_filename    = "../output/decay_M49_NFW2D_FOVdiameter14.0deg_rse1_alphaint0.06deg_nside2048.fits"
VirgoC_filename = "../output/decay_VirgoC_NFW2D_FOVdiameter14.0deg_rse1_alphaint0.06deg_nside2048.fits"
"""
M87_filename    = "../output/decay_M87_GAO2D_FOVdiameter14.0deg_rse1_alphaint0.06deg_nside2048.fits"
M49_filename    = "../output/decay_M49_GAO2D_FOVdiameter14.0deg_rse1_alphaint0.06deg_nside2048.fits"
VirgoC_filename = "../output/decay_VirgoC_GAO2D_FOVdiameter14.0deg_rse1_alphaint0.06deg_nside2048.fits"
"""
M87_hdus = fits.open(M87_filename)
M49_hdus = fits.open(M49_filename)
Virgo_hdus = fits.open(VirgoC_filename)

for i in range(1, len(M87_hdus)):
    print("header {}: {}".format(i, M87_hdus[i].header['EXTNAME']))
#    
#for i in range(1, len(M49_hdus)):
#    print("header {}: {}".format(i, M49_hdus[i].header['EXTNAME']))

header 1: JFACTOR
header 2: JFACTOR_PER_SR


In [3]:
def generate_cartesian_file(hdus, lon, lat, label, index=1):
    content = hdus[index].header
    theta_0 = content['THETA_0']
    psi_0   = content['PSI_0']
    size_x  = content['SIZE_X']
    size_y  = content['SIZE_Y']
    dangle  = content['ALPHAINT']

    content = hdus[index].data
    pixels  = content['PIXEL']
    Jtotal  = content['Dtot']
    Jsmooth = content['Dsmooth']
    Jsub    = content['Dsub']
    #Jcross  = content['Dcrossp']
    NSIDE    = hdus[2].header['NSIDE']
    ordering = hdus[2].header['ORDERING']
    if ordering == "NESTED":
        nested = True
    else:
        nested = False

    nPix = hp.nside2npix(NSIDE)
    num_used_pixels = len(pixels)
    theta_rad, phi_rad = hp.pixelfunc.pix2ang(NSIDE, pixels, nest=nested)
    # change the angles according to our conventions
    theta, phi = -np.degrees(theta_rad-np.pi/2.-lon), np.degrees(np.pi*2.-phi_rad+lat)
    phi[np.where(phi>360)] -= 360.
    # Process the data for 3ML use
    nxPix = 140
    nyPix = 140
    refX  = 70.5
    refY  = 70.5
    delX  = -dangle
    delY  = dangle
    x = np.zeros(nPix)
    x[pixels] = Jtotal/np.max(Jtotal)
    dmROI = np.zeros([nxPix, nyPix])
    for i in range(nxPix):
        for j in range(nyPix):
            ra_roi = (i-np.int(refX))*delX                                                                          
            dec_roi = (j-np.int(refY))*delY
            hpix = hp.pixelfunc.ang2pix(NSIDE,np.radians(-dec_roi+90.),np.radians(360.-ra_roi), nest=nested)
            #print(hpix)
            dmROI[i,j] = x[hpix]
    dmROI = np.multiply(dmROI, (np.degrees(1)**2/delX**2))
    # convert from galactic coordinates to fk5
    coords = SkyCoord(lon*u.degree, lat*u.degree, frame='galactic', unit='degree')
    RA  = coords.fk5.ra.degree
    DEC = coords.fk5.dec.degree
    # write the output to the new file
    new_hdu = fits.PrimaryHDU(dmROI)
    new_hdu.header['CTYPE1'] = 'RA'
    new_hdu.header['CTYPE2'] = 'DEC'
    new_hdu.header['CUNIT1'] = 'deg'
    new_hdu.header['CUNIT2'] = 'deg'
    new_hdu.header['CRPIX1'] = refX
    new_hdu.header['CRPIX2'] = refY
    new_hdu.header['CRVAL1'] = RA
    new_hdu.header['CRVAL2'] = DEC
    new_hdu.header['CDELT1'] = delX
    new_hdu.header['CDELT2'] = delY
    #new_hdu.header['DMAX']   = np.log10(Jtotal.max())
    hdulist = fits.HDUList([new_hdu])
    hdulist.writeto('{}_Dfactor_template.fits'.format(label))
    print("For {}: Dmax={}".format(label, np.log10(Jtotal.max())))

In [4]:
generate_cartesian_file(M87_hdus, lon_M87, lat_M87, "M87_nfw")
generate_cartesian_file(M49_hdus, lon_M49, lat_M49, "M49_nfw")
generate_cartesian_file(Virgo_hdus, lon_Virgo, lat_Virgo, "VirgoC_nfw")

For M87_nfw: Dmax=18.0183296204
For M49_nfw: Dmax=17.6360435486
For VirgoC_nfw: Dmax=18.6541938782
